## Introduction
#### Accessing data is first step to any data analysis step.
#### Input and Output fall in a few main categories: reading tabular data or other on-disk formats, loading data from databases, and interacting with network sources like web APIs.

## Reading and Writing Data in Text Format
#### pandas has number of functions for reading tabular data as DataFrame object. 'read_csv' and 'read_table' are most likely to be used.
#### # All the functions in general convert text data into a DataFrame. The optional arguments in these functions can fall into a few categories:
#### 1. Indexing - Treat 1 or more columns as returned DataFrame, To get column names from the file, the user or not at all.
#### 2. Type Inference and Data Conversion - Converting user-defined values and custom list of missing value markers.
#### 3. Datetime Parsing - Combining date and time information from multiple columns into a single one.
#### 4. Iterating - Support for iterating over chunks of data of very large files.
#### 5. Unclean Data Issues - Skipping rows or footer, comments, or other minor things.

#### Because real-world data is messy, some of the data loading functions have become very complex over time.
#### Some of the functions have 'type inference' as column data type is not a part of the data format. So we don't have to specify which columns are numeric, text, etc.
#### Some formats (HDF5, Feather, msgpack) have data types already in their format.
#### Handling dates and other custom data types require more effort.

In [6]:
!type examples\ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


#### If the file is comma-delimited, we can just use read_csv to read it.
#### We can also use read_table with specific delimiter (here ,).

In [8]:
import pandas as pd

df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
pd.read_table('examples/ex1.csv',sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### A file will not always have a header row. For this we can either allow pandas to assign default names or specify the names ourselves.

In [11]:
! type examples\ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [12]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
pd.read_csv('examples/ex2.csv', names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### If you want a specific column to be taken as index (here message) of the DataFrame, you can specify the Column name in the 'index_col' parameter. Or specify the index number of the column.
#### To form hierarchical index from multiple columns, pass a list of column numbers or names in that order.

In [15]:
names = ['a','b','c','d','message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [16]:
!type examples\csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [18]:
parsed = pd.read_csv('examples/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

#### In some cases, input may not have same delimiter. We may need to whitespace or some patter in this case.
#### If the variables have variable amount of whitespace, we can use regex (here '\s+') as delimiter for read_table.
#### The read_table function infers by the number of columns present in the first row that the first row is header.

In [20]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [23]:
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


#### Parser function can have multiple arguments to handle wide variety of exception file formats.
#### We can skip rows in a file with 'skiprows'.

In [25]:
!type examples\ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [26]:
pd.read_csv('examples/ex4.csv', skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### Missing data from a file is usuall not present (empty string) or marked by 'sentinel value'.
#### pandas by default recognizes the commonly occuring sentinel values like NA and NULL and replaces them with NaN.
#### We also have the 'na_values' option to have a list or set of strings that can be considered as missing values.
#### We can have different sentinel lists for different columns through a dictionary.

In [28]:
!type examples\ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [29]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [30]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [31]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [32]:
sentinels = {'message': ['foo','NA'], 'something':['two']}

pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in Pieces
#### Before processing a very large file or figuring out the right way to correctly process a file, we may need to read small piece of the file or iterate over smaller chunks of the file.
#### To look at a large file, we can compact the display rows by setting 'max_rows'.
#### For the same reason, we can select a smaller number of rows by using the 'nrows' parameter in read_csv.

In [34]:
pd.options.display.max_rows = 8

result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
...,...,...,...,...,...
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G
9999,-0.096376,-1.012999,-0.657431,-0.573315,0


In [35]:
pd.read_csv('examples/ex6.csv',nrows=4)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R


#### To read a file in pieces, specify the 'chunksize' in the argument.
#### The object returned from such a read is a TextParser object and it allows you to iterate over parts of the file decided by chunksize.
#### TextParser also has a get_chunk method that lets you read pieces of arbitrary size.

In [37]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=100)
chunker

In [38]:
tot = pd.Series([])

for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
     ...  
J    337.0
F    335.0
K    334.0
H    330.0
Length: 10, dtype: float64

### Writing Data to Text Format
#### Data can be exported to delimited format. The 'to_csv' method can write data to a comma-seperated file.
#### Other delimiters can also be used with the 'sep' argument. We can display result on console using the 'sys.stdout' argument.
#### Missing values appear as empty string in the oputput. We can denote them with a different sentinel value with 'na_rep' argument.

In [41]:
data = pd.read_csv('examples/ex5.csv')

data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [42]:
data.to_csv('examples/out.csv')

In [43]:
!type examples\out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [44]:
import sys

data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [45]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


#### By default, both row and column headers are written. But they can be disabled with the 'index=False' and 'header=False' argument for rows and columns respectively.
#### We can can also write a subset of the columns in the order we want.
#### Series also has the to_csv method.

In [47]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [51]:
import numpy as np

dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('examples/tseries.csv')

In [53]:
!type examples\tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### Working with Delimited Formats
#### We can load most forms of tabular data from disk using functions like pandas.read_table.
#### But in some cases, manual processing may be necessary. Its quite common to get files with 1 or more malformed lines that trip up the usual functions.
#### Let's use a small CSV file for illustration.

In [55]:
!type examples\ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


#### For a file with single character delimiter, we can use built-in csv module. We can open a file or file-like object to csv.reader.
#### The output would be a tuple of values with quotes removed.
#### From here we can wrangle the data in whichever way we need it.

In [58]:
import csv

f = open('examples/ex7.csv')
reader = csv.reader(f)

In [59]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


#### First we will read the file into a list of lines.
#### Then we split the lines into header and data lines.
#### We can then create a dictionary of data columns using dictionary comprehension and the expression 'zip(*values)'. This transposes rows into columns.

In [61]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [62]:
header, values = lines[0], lines[1:]

In [64]:
data_dict = {h: v for h,v in zip(header,zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

#### CSV files come in different flavours.
#### To define a new format with different delimiter, quotation convention, or line terminator, just define a simple subclass of 'csv.Dialect'.
#### we can also give individual CSV dialect parameters as keywords to csv.reader.

In [66]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [69]:
f = open('examples/ex7.csv')
reader = csv.reader(f, dialect=my_dialect)

for line in reader:
    print(line)

['a,"b","c"']
['1,"2","3"']
['1,"2","3"']


In [71]:
f = open('examples/ex7.csv')
reader = csv.reader(f, delimiter='|')
for line in reader:
    print(line)

['a,"b","c"']
['1,"2","3"']
['1,"2","3"']


#### To write delimited files manually, use 'csv.writer'. 
#### It accepts open, writable file object and same dialect and format options as csv.reader.
#### NOTE - For more complicated formats, you will not be able to use csv module. In this case you will have to use methods like String split or re.split and do the cleanup manually.

In [74]:
with open('examples/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [75]:
!type examples\mydata.csv

one;two;three
1;2;3
4;5;6
7;8;9


### JSON Data
#### JSON (JavaScript Object Notation) has become one of the standard formats for sending data via HTTP between browsers and applications.
#### It is a much more free-form data format than CSV.
#### It is very valid in Python except for a few exceptions:
####     1. Its 'null' value that is not recognized in Python.
####     2. Disallowing trailing commas at end of list.
    
#### The basic types in JSON are:
####     1. Objects (dicts)
####     2. Arrays (lists)
####     3. Strings
####     4. Numbers
####     5. Booleans
####     6. Nulls
#### All the keys in the object must be strings.

#### There are several libraries for working with JSON files. One of them is 'json', which is built into standard library.
#### We can use 'json.loads' to convert JSON string to Python DataFrame.
#### 'json.dumps' converts Python object back to JSON.
#### To convert a JSON object or list of objects to a DataFrame can be conviniently done by passing the list to DataFrame constructor and select subset of data fields.

In [78]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [79]:
import json

result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [82]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [83]:
siblings = pd.DataFrame(result['siblings'], columns=['name','age'])
siblings

,name,age
0,Scott,30
1,Katie,38


#### 'pandas.read_json' can automatically convert JSON datasets into Series or DataFrame.
#### The default options for this assume that each object in the JSON array is a row in the table.
#### To export data from pandas to JSON, we can use 'to_json' method on a Series or DataFrame.

In [86]:
!type examples\example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [87]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [88]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [89]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### XML and HTML : Web Scraping
#### There a few libraries for reading and writing data in the ubiquitous HTML and XML formats. Eg - lxml, Beautiful Soup, html5lib.
#### lxml is comparatively faster in general. Other libraries handle malformed data better.
#### pandas has a function 'read_html' which uses lxml and Beautiful Soup under the hood and automatically parses tables out of HTML files as DataFrame objects.
#### By default it searches for and attempts to parse tabular data contained within 'table' tags.
#### After getting the XML or HTML data into a DataFrame, we could perform data cleaning and analysis.

In [3]:
import pandas as pd

tables = pd.read_html('examples/fdic_failed_bank_list.html')
len(tables)

1

In [4]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [5]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

### Parsing XML with lxml.Objectify
#### XML (eXtensible Markup Language) is a common structured data format supporting hierarchical, nested data with metadata.
#### XML and HTML are structurally similar, but XML is more general.
#### Lets take for example the performance data for each train or bus service put up by New York Metropoliton Authority (MTA).

In [9]:
!type datasets\mta_perf\Performance_MNR.xml

<?xml  version="1.0" encoding="ISO-8859-1"?>
<PERFORMANCE>
<INDICATOR>
  <INDICATOR_SEQ>28445</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>On-Time Performance (West of Hudson)</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.
</DESCRIPTION>
  <PERIOD_YEAR>2008</PERIOD_YEAR>
  <PERIOD_MONTH>1</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>95.00</YTD_TARGET>
  <YTD_ACTUAL>96.90</YTD_ACTUAL>
  <MONTHLY_TARGET>95.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>96.90</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICAT

  <YTD_ACTUAL>97.80</YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>97.20</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28461</INDICATOR_SEQ>
  <PARENT_SEQ>28445</PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Pascack Valley Line - OTP</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. Metro-North Railroad contracts with New Jersey Transit to operate service on the Pascack Valley Line.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>1</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL>94.90</YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>94.90</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICAT

  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28530</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Customer Injury Rate</INDICATOR_NAME>
  <DESCRIPTION>Any injury to a customer as a result of an incident within/on railroad property. Included are injuries that result from an assault or crime. The rate is injuries per million customers.</DESCRIPTION>
  <PERIOD_YEAR>2012</PERIOD_YEAR>
  <PERIOD_MONTH>4</PERIOD_MONTH>
  <CATEGORY>Safety Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>D</DESIRED_CHANGE>
  <INDICATOR_UNIT>-</INDICATOR_UNIT>
  <DECIMAL_PLACES>2</DECIMAL_PLACES>
  <YTD_TARGET></YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET></MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28530</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Customer Injury Ra

  <YTD_TARGET>97.90</YTD_TARGET>
  <YTD_ACTUAL>97.90</YTD_ACTUAL>
  <MONTHLY_TARGET>97.90</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>98.10</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28345</INDICATOR_SEQ>
  <PARENT_SEQ>55526</PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Hudson Line - OTP</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time.</DESCRIPTION>
  <PERIOD_YEAR>2009</PERIOD_YEAR>
  <PERIOD_MONTH>11</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.90</YTD_TARGET>
  <YTD_ACTUAL>97.90</YTD_ACTUAL>
  <MONTHLY_TARGET>97.90</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>97.90</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28345</INDICATOR_SEQ>
  <PARENT_SEQ>55526</PARENT_SEQ>
  <AGENCY_NAME

  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.60</YTD_TARGET>
  <YTD_ACTUAL>97.70</YTD_ACTUAL>
  <MONTHLY_TARGET>97.60</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>98.30</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>55526</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>On-Time Performance (East of Hudson)</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. East of Hudson service includes the Harlem, Hudson and New Haven lines.</DESCRIPTION>
  <PERIOD_YEAR>2009</PERIOD_YEAR>
  <PERIOD_MONTH>6</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.60</YTD_TARGET>
  <YTD_ACTUAL>97.70</YTD_ACTUAL>
  <MONTHLY_TARGET>97.60</MONTHLY_T

#### Using lxml we parse the file and get reference to root node of the XML file with 'getroot'.
#### 'root.INDICATOR' returns a generator yielding each INDICATOR tag XML element. For each record, we can have dict of tag names to data values.
#### Lastly, we get a list of dicts into a DataFrame.

In [12]:
from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [15]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)
        

In [16]:
perf = pd.DataFrame(data)

perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


#### XML data can get much more complicated. Each tag can have metadata too. Eg - HTML link tag, which also contains valid XML.
#### We can access any of the fields in th tag or the link text.

In [18]:
from io import StringIO

tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [19]:
root

<Element a at 0x21e29242388>

In [20]:
root.get('href')

'http://www.google.com'

In [21]:
root.text

'Google'

## Binary Data Formats
#### The easiest way to store data called 'serialization' uses efficient binary formatting. Eg - pickle in Python.
#### pandas objects have 'to_pickle' method that writes them into a pickle format.
#### We can read a pickled object by using pickle directly or conviniently using 'pandas.read_pickle'.
#### NOTE - pickle is only recommended for short-term storage format. The format may not be stable over time. An object pickled today may not be un-pickled in a later version.

In [23]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [24]:
frame.to_pickle('examples/frame_pickle')

In [25]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### pandas has built-in support for 2 more binary format: HDF5 and MessagePack.
#### Some other storage formats for pandas and NumPy are:
####     1. bcolz - Compressible column-oriented binary format based on Blosc compression library.
####     2. Feather - Cross-language column-oriented binary format. Uses Apache Arrow columnar memory format.

### Using HDF5 Format
#### It is a well-regarded file format for storing large quantities of scientific array data.
#### It is avalilable in the C library and has interfaces in other languages like Java, Julia, MATLAB and Python.
#### HDF stands for Hierarchical Data Format. Each file can stor multiple datasets and supporting metadata.
#### It supports on-the-fly compression with other compression modes, allowing data with repeated patterns to be stored more efficiently.
#### It is a good choice for working with very large datasets that don't fit in memory, and can be efficiently read and written in small sections.

#### It is possible to access HDF5 file either through PyTables or h5py libraries.
#### pandas allows high-level interface that simplifies storing pandas objects. The HDFStore class works like a dict and handles low-level details.
#### Objects inside the HDF5 file can then be retrieved with same dict-like API.

In [3]:
import numpy as np
import pandas as pd

frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('examples/mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']

In [4]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: examples/mydata.h5

In [6]:
store['obj1'].head()

,a
0,-1.261068
1,1.752054
2,-0.551721
3,-0.018368
4,0.316582


#### HDFStore supports 2 storage schemas - 'fixed' and 'table'. table is generally slower but supports query operations using special syntax.
#### The 'put' is an explicit version of "store['obj2'] = frame" method but allows other options in the storage format.
#### The 'pandas.read_hdf' function gives you shortcut to these tools.

In [8]:
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,-0.007719
11,1.207900
12,-0.515168
13,0.457047
14,-0.782020
15,0.657464


In [9]:
store.close()

In [10]:
frame.to_hdf('mydata.h5', 'obj3', format='table')
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,-1.261068
1,1.752054
2,-0.551721
3,-0.018368
4,0.316582


#### NOTE - For storage in remote servers like Amazon S3 or HDFS, use a different format suitable for distributed storage like Apache Parquet.
#### Caution - HDF5 is not a database. It is suited for 'write once, read many' datasets. Although data can be added to a file anytime, but if multiple writers do it simultaneously, the file becomes corrupted.

### Reading Microsoft Excel Files
#### pandas also supports reading files in Excel 2003 and higher using either the 'ExcelFile' class or 'pandas.read_excel' function.
#### Internally these tools use add-on packages 'xlrd' or 'openpyxl' to read the XLS or XSSX files respectively.

#### To use ExcelFile, create an instance by passing a path to an xls or xlsx file.
#### Data stored in a sheet can be read into a DataFrame with 'parse'.

In [13]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [14]:
pd.read_excel(xlsx, 'Sheet1')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### To read multiple sheets in a file, it is faster to create the ExcelFile, but we can simply pass the filename to 'pandas.read_excel'.
#### To write to Excel format, first create ExcelWriter, then write data to it using pandas 'to_write' method.
#### To avoid ExcelWriter, pass a file path to to_excel.

In [16]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [17]:
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [18]:
frame2 = pd.read_excel('examples/ex2.xlsx', 'Sheet1')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [19]:
frame.to_excel('examples/ex2.xlsx')

### Interacting with Web APIs
#### Many websites have public APIs that provide data feeds via JSON or some other format.
#### There are a number of ways to get their data, one of which is the 'requests' package.
#### In this example, we will find last 30 Github issues for pandas. We will use 'GET HTTP' request using requests library.
#### The response object's JSON will provide a dictionary having JSON parsed into native Python objects.
#### Each element in 'data' is a dict containing all of the data found in the Github Issue page (except for comments).
#### From this we can pass data directly to a DataFrame and extract fields of interest.

In [21]:
import requests

url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [23]:
data = resp.json()
data[0]['title']

'pd.to_timedelta not parsing iso-formatted strings'

In [25]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
issues.head()

,number,title,labels,state
0,21877,pd.to_timedelta not parsing iso-formatted strings,[],open
1,21874,BUG: Align Series.str.zfill() with str.zfill(),"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
2,21873,TST: Parameterize more tests,"[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open
3,21872,"[REF] Move comparison methods to EAMixins, sha...","[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open
4,21871,API: Add DataFrame.droplevel,"[{'id': 35818298, 'node_id': 'MDU6TGFiZWwzNTgx...",open


#### With some more work, we can create our own high-level interfaces to common web APIs that return DataFrame objects for easy analysis.

## Interacting with Databases
#### SQL Based relational Databases are widely used  to store data. Eg - SQL Server, PostgreSQL, MySQL, etc.
#### Many alternative databases have also become quite popular.
#### The choice of DataBase is usually dependant on performance, data integrity and scalability nneds of the application.

#### Loading data from SQl to DataFrame is straightforward. pandas has some functions to simplify the process.
#### In this example, we will create a SQLite database using Python's built in sqlite3 driver.

In [31]:
import sqlite3

query = """
CREATE TABLE test1
(a VARCHAR(20), b VARCHAR(20),
c REAL, d INTEGER);
"""

con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [32]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO test1 VALUES(?,?,?,?)"

con.executemany(stmt, data)

In [33]:
con.commit()

#### Most SQL Drivers (PyODBC, psycopg2, MySQLdb, pymssql, etc.) return a list of tuples when selecting data from table.
#### We can use these list of tuples for the DataFrame, but the column names are present in the cursor's 'description' attribute.

In [36]:
cursor = con.execute('select * from test1')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [37]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [38]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


#### This is a lot of data munging taht we don't want to repeat.
#### The 'SQLAlchemy' is a popular Python toolkit that abstracts away many common differences between SQL databases.
#### pandas has 'read_sql' function that enables us to read data easily from general SQLAlchemy connection.

In [40]:
import sqlalchemy as sqla

db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test1', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
